In [3]:
# ============================================
# 1) 설치
# ============================================
!pip -q install langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers pypdf


In [ ]:

# ============================================
# 2) PDF 다운로드
# ============================================
import os, urllib.request, requests

url = "https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf"
pdf_path = "2020_경제금융용어 700선_게시.pdf"

try:
    urllib.request.urlretrieve(url, pdf_path)
except Exception:
    pass

if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0:
    r = requests.get(url, stream=True, timeout=60)
    r.raise_for_status()
    with open(pdf_path, "wb") as f:
        for chunk in r.iter_content(8192):
            if chunk:
                f.write(chunk)
print("✅ PDF ready:", pdf_path)

# ============================================
# 3) PDF 로드 & 청크 분할
# ============================================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"📄 페이지 수: {len(docs)} | 🧩 청크 수: {len(chunks)}")
print("샘플:", chunks[0].page_content[:150].replace("\n"," "))

# ============================================
# 4) 1차 검색: 한국어 임베딩 + FAISS
# ============================================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
faiss_store = FAISS.from_documents(chunks, embedding=embedding)

# 재랭킹 여지를 주기 위해 후보를 넉넉히 뽑습니다.
first_stage = faiss_store.as_retriever(search_type="similarity", search_kwargs={"k": 30})

# ============================================
# 5) 2차 재랭킹: Cross-Encoder (BAAI/bge-reranker-base)
# ============================================
from sentence_transformers import CrossEncoder
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 공개 모델 (한국어 포함 다국어에 강함)
# 더 강한 모델: "BAAI/bge-reranker-large" (속도/메모리↑)
reranker = CrossEncoder(
    "BAAI/bge-reranker-base",
    device=device,
    max_length=512
)

def cross_encoder_rerank(query, docs, top_k=5, batch_size=32):
    pairs = [(query, d.page_content) for d in docs]
    scores = []
    for i in range(0, len(pairs), batch_size):
        with torch.inference_mode():
            s = reranker.predict(pairs[i:i+batch_size])
        # numpy -> list
        scores.extend(s.tolist() if hasattr(s, "tolist") else list(s))
    scored = list(zip(docs, scores))
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:top_k]

# ============================================
# 6) 통합 질의 함수 (1차 검색 → 2차 재랭킹)
# ============================================
def ask(query, top_k_first=30, top_k_final=5, show_scores=True):
    candidates = first_stage.invoke(query)
    reranked = cross_encoder_rerank(query, candidates, top_k=top_k_final)
    print(f"\n🔎 Q: {query}")
    print(f"  1차 후보 {len(candidates)}개 → 2차 상위 {len(reranked)}개")
    for i, (d, s) in enumerate(reranked, 1):
        page = d.metadata.get("page", "?")
        preview = d.page_content[:160].replace("\n"," ")
        if show_scores:
            print(f"{i:>2}. p.{page} | score={s:.4f} | {preview}...")
        else:
            print(f"{i:>2}. p.{page} | {preview}...")

# ============================================
# 7) 테스트 질의
# ============================================
ask("인플레이션의 정의와 원인, 그리고 기준금리와의 관계는 무엇인가요?")
ask("환율 변동이 물가에 어떤 영향을 미치나요?")
ask("스태그플레이션의 의미와 정책 대응상의 어려움은 무엇인가요?")


✅ PDF ready: 2020_경제금융용어 700선_게시.pdf
📄 페이지 수: 371 | 🧩 청크 수: 1117
샘플: iii 찾아보기 한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인  의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠  개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 .  이의 일환으로 2018년 
Device: cpu


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

/tmp/ipython-input-2211192549.py:82: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  candidates = first_stage.get_relevant_documents(query)



🔎 Q: 인플레이션의 정의와 원인, 그리고 기준금리와의 관계는 무엇인가요?
  1차 후보 30개 → 2차 상위 5개
 1. p.356 | score=0.2076 | 대해서는 시대에 따라 변해왔다. 보다 긴 시각에서 본다면 중앙은행은 대체로 가격지표 인 금리를 중시한 통화정책을 수행해왔다고 평가된다. 하지만 지난 1970년대 석유파동  등으로 높은 인플레이션을 겪는 가운데 각국은 물가안정을 위해 양적인 지표인 통화량을  보다 중시하고 통화량을 안정적...
 2. p.131 | score=0.1018 | 115 ㅁ  물가지수 시장에서 거래되는 여러 가지 상품과 서비스의 가격을 경제생활에서 차지하는 중요도 를 고려하여 평균한 종합적인 가격수준을 물가라고 하는데, 이 같은 물가의 변화를  한 눈에 알아볼 수 있도록 기준연도의 물가수준을 100으로 놓고 비교되는 다른 시점의  물가를 지수의 ...
 3. p.86 | score=0.0789 | Bureau of Economic Research)에서 발간한 경기변동관련 연구로부터 유래한 것으로  추정된다.  연관검색어 : 경기, 경기종합지수, 동행종합지수 기준환율  기준환율이란 일반적으로 “자국 통화와 여러 외국 통화간의 환율결정에서 다른 외국  통화 환율 결정의 기준이 되는 ...
 4. p.113 | score=0.0654 | 2.0% 등으로 낮아지는 경우 매년 물가상승률이 0보다 크기 때문에 상품과 서비스의  가격 수준은 지속적으로 상승하고 있지만, 상승률은 5.0%→3.5%→2.0%로 낮아지고  있는데 바로 이러한 현상을 디스인플레이션(disinflation)이라 한다. 디스인플레이션은  단기간에 그치면 ...
 5. p.322 | score=0.0361 | (accountability)을 다하고 있다 . 통화정책 파급경로 통화정책이 실물경제에 파급되는 경로에는 금리경로, 자산가격경로, 환율경로, 기대경 로, 신용경로, 위험선호경로 등이 있다. 금리경로는 중앙은행이 정책금리를 인하하면  단기시장금리와 함께 장기시